# 1. Lecture Overview

In this lecture, we apply some of the main tools we covered so far, to prepare the CRSP and Compustat datasets for analysis for the rest of the course. 

In the Data Science community, this process of transforming a "raw" dataset into one that is more appropriate for downstream analysis is also called **data wrangling** or **data munging**. 

We will save these processed datasets for later use, so we don't have to start from scratch every time we need to use the CRSP and Compustat data later on.

- Compustat
    - Cleaning
    - Building some key variables
    
    
- CRSP
    - Cleaning
    - Building some key variables

# 2. Preliminaries

In [ ]:
# Import libraries
import pandas as pd
import datetime as dt

# Pretty print all cell's output and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 3. Compustat application

- Load Compustat data (compa.zip), convert all names to lowercase, rename lpermno to permno and datadate to date
- Create the datetime date (dtdate), the annual date (ydate) and set the index for the dataset (permno ydate)
- Clean the data
    - drop the following columns: 'indfmt', 'consol', 'popsrc', 'curcd', 'datafmt','costat'
    - drop observations with negative assets (at) or sales (at)
    - keep only firms incorporated in the US (fic == USA)
- Create new variables
    - capx2assets = capx divided by lagged total assets
    - roa = ib divided by lagged total assets
    - bleverage = (dlc + dltt) divided by total assets
    - cash2assets = che divided by total assets
    - cflow2assets = (ib + dp) divided by lagged total assets
    - mktcap = prcc_f * csho
- Winsorize the new variables at the 1 and 99 percentiles    
- Save new variables in a new dataset called "compa_clean" (in a tab-delimited txt file)       

### Load data and clean variable names

In [ ]:
# Load Compustat data (compa.zip)
comp = pd.read_csv('./compa.zip', sep = '\t', low_memory = False)

comp.head()
comp.dtypes

In [ ]:
# Convert all column names to lowercase
comp.columns = comp.columns.str.lower()
comp.dtypes

In [ ]:
# Rename lpermno to permno and datadate to date
comp.rename(columns = {'lpermno':'permno','datadate':'date'}, inplace = True)
comp.dtypes

### Create necessary date variables

In [ ]:
# Create the datetime date (dtdate)
comp['dtdate'] = pd.to_datetime(comp['date'], format = '%Y%m%d')

# Print just 'dtdate' and 'date' columns
comp[['date','dtdate']]

In [ ]:
# Create the annual date (ydate)
comp['ydate'] = comp['dtdate'].dt.to_period('Y')

# Print all three date columns
comp[['date','ydate','dtdate']]

### Impose filters (deleting some rows and/or some columns)

In [ ]:
# Find initial number of observations (rows)
n_raw = len(comp)
print(n_raw)

In [ ]:
# Drop the following columns: 'indfmt', 'consol', 'popsrc', 'curcd', 'datafmt','costat'
comp.drop(columns = ['indfmt', 'consol', 'popsrc', 'curcd', 'datafmt','costat'], inplace = True)
comp.dtypes #this gives us a full list of the remaining columns so we can check that the above have been deleted

In [ ]:
# Drop observations with negative assets (at) or sales (at)
comp = comp.loc[(comp['at']>0) & (comp['sale']>0)].copy()

# Calculate how many rows we have left after this step and print it out 
n_after1 = len(comp)
print(n_raw, n_after1, n_raw - n_after1)

In [ ]:
# Keep only firms incorporated in the US (fic == USA)  
comp = comp.loc[comp['fic']=='USA'].copy()

# Calculate how many rows we have left after this step and print it out 
n_after2 = len(comp)
print(n_after1, n_after2, n_after1 - n_after2)

### Set the index of the dataset

In [ ]:
# Set the index
comp.set_index(['permno','ydate'], inplace = True, drop = False)
comp.head()
comp.dtypes

In [ ]:
# Drop duplicates in the index (firms with multiple observations in a given year)
comp = comp.loc[~comp.index.duplicated(keep=False)].copy()

# Calculate how many rows we have left after this step and print it out 
n_after3 = len(comp)
print(n_after2, n_after3, n_after2 - n_after3)

### Create new variables

In [ ]:
# lagged total assets
comp['at_lag1'] = comp.groupby(level='permno')['at'].shift(1)

# print results
comp[['at','at_lag1']]

In [ ]:
# capx2assets = capx divided by lagged total assets
comp['capx2assets'] = comp['capx'] / comp['at_lag1']

# print results
comp[['capx','at_lag1', 'capx2assets']]

In [ ]:
# roa = ib divided by lagged total assets
comp['roa'] = comp['ib'] / comp['at_lag1']

# print results
comp[['ib','at_lag1', 'roa']]

In [ ]:
# bleverage = (dlc + dltt) divided by total assets
comp['bleverage'] = (comp['dlc'] + comp['dltt']) / comp['at']

# print results
comp[['dlc','dltt', 'at', 'bleverage']]

In [ ]:
# cash2assets = che divided by total assets
comp['cash2assets'] = comp['che'] / comp['at']

# print results
comp[['che','at', 'cash2assets']]

In [ ]:
# cflow2assets = (ib + dp) divided by lagged total assets
comp['cflow2assets'] = (comp['ib'] + comp['dp']) / comp['at_lag1'] 

# print results
comp[['ib','dp', 'at_lag1', 'cflow2assets']]

In [ ]:
# mktcap = prcc_f * csho
comp['mktcap'] = comp['prcc_f'] * comp['csho']

# print results
comp[['prcc_f','csho', 'mktcap']]

#### Example
Create a new variable in 'comp' called 'sg5' that equals the percentage growth in sales over the past 5 years (current year vs five years ago). 

In [ ]:
# Create column with sales lagged 5 years
comp['sale_lag5'] = comp.groupby(level = 'permno')['sale'].shift(5)

#Calculate sales growth
comp['sg5'] = comp['sale']/comp['sale_lag5'] - 1

In [ ]:
# Print results
comp.loc[25881,['sale','sale_lag5','sg5']]

### Winsorize the new variables

"Winsorizing" a variable at the 1 and 99 percentiles, we mean that the values of that variable that are smaller than the 1st percentile are made equal to the 1st percentile and the values that are larger than the 99th percentile are made equal to the 99th percentile.

You can pick other values for the percentiles at which you want to winsorize but (1,99) and (5, 95) are by far the most common ones.

To winsorize a variable, in a Pandas dataframe, we use the **clip** function as below. This also requires us to use the **quantile** function to calculate the 1 and 99th percentiles.

In [ ]:
# Calculate the 1st and 99th percentiles of 'roa'
roa_p1 = comp['roa'].quantile(0.01)
roa_p99 = comp['roa'].quantile(0.99)

print(roa_p1, roa_p99)

In [ ]:
# Winsorize 'roa' at 1st and 99th percentiles
comp['roa_w'] = comp['roa'].clip(lower = roa_p1, upper = roa_p99)

# Take a look at roa and roa_w
comp[['roa','roa_w']]

In [ ]:
# Winsorize the all new variables at the 1 and 99 percentiles 
for var in ['capx2assets', 'roa', 'bleverage', 'cash2assets', 'cflow2assets', 'mktcap']:
    comp[f'{var}_w'] = comp[var].clip(lower = comp[var].quantile(0.01), upper = comp[var].quantile(0.99))

In [ ]:
# Take a quick look at the winsorized variables        
comp.filter(like = '_w', axis = 1) 

### Save the new dataset in a pickle file

In [ ]:
# Save new variables in a new dataset called "compa_clean" (in pickle file file)  
comp.to_pickle('./comp_clean.pkl')

# Remember, you can read this file with "read_pickle"
comp = pd.read_pickle('./comp_clean.pkl')
#comp

# 4. CRSP application

- Load CRSP file (crspm.zip), convert all names to lowercase
- Create monthly date variable (mdate) and set the index for the dataset (permno mdate)
- Replace stock price (prc) with its absolute value
- Impose filters
    - keep only observations where ret is numeric
    - keep only observations where exchcd is 1, 2, or 3
    - keep only observations where shrcd is 10 or 11
- Create new variables
    - mktcap = price (prc) times number of shares (shrout)
    - ret12 = compounded returns over past 12 months
- Save ret and mktcap to a new dataset called "crsp_clean" (in a tab-delimited txt file)    

### Load data, clean prices, and create date variables

In [ ]:
# Load CRSP file (crspm.zip)
crsp = pd.read_csv('./crspm.zip', sep = '\t', low_memory = False)

# Take a look at the dataset
crsp
crsp.dtypes

In [ ]:
# Convert all column names to lowercase
crsp.columns = crsp.columns.str.lower()

# Take a look at the dataset
crsp.dtypes

In [ ]:
# Replace prc with its absolute value
crsp['prc'] = crsp['prc'].abs()

# Take a look at the prc column
crsp['prc']

In [ ]:
# Create datetime date variable 
crsp['dtdate'] = pd.to_datetime(crsp['date'], format = '%Y%m%d')

# Take a look at the date variables
crsp[['date','dtdate']]

In [ ]:
# Create period date variable (monthly frequency)
crsp['mdate'] = crsp['dtdate'].dt.to_period('M')

# Take a look at the date variables
crsp[['date','dtdate', 'mdate']]

In [ ]:
# Another way to look at all the variables containing "date" in their name
crsp.filter(like = "date")

### Impose filters

In [ ]:
# Check initial number of observations (rows)
n_raw = len(crsp)
print(n_raw)  

In [ ]:
# Keep only observations where ret is numeric
    # Keep only rows where to_numeric() does not fail on 'ret' column
crsp = crsp.loc[pd.to_numeric(crsp['ret'],errors='coerce').notnull()].copy()  
crsp
    # Convert 'ret' column to numeric type
crsp['ret'] = pd.to_numeric(crsp['ret'])

# See how many observations were lost after these steps
n_after1 = len(crsp)
print(n_raw, n_after1, n_raw-n_after1)

In [ ]:
# Keep only observations where exchcd is 1, 2, or 3
crsp = crsp.loc[crsp['exchcd'].isin([1,2,3])].copy()

# See how many observations were lost after these steps
n_after2 = len(crsp)
print(n_after1, n_after2, n_after1 - n_after2)

In [ ]:
# Keep only observations where shrcd is 10 or 11
crsp = crsp.loc[crsp['shrcd'].isin([10,11])].copy()

# See how many observations were lost after these steps
n_after3 = len(crsp)
print(n_after2, n_after3, n_after2 - n_after3)

In [ ]:
# Drop date, exchcd, shrcd
crsp.drop(columns = ['date', 'exchcd', 'shrcd'], inplace = True)    

# Take a look at all the variables left in the dataset
crsp.dtypes

### Set the index

In [ ]:
# Set the index for the dataset (permno mdate)
crsp.set_index(['permno', 'mdate'], inplace = True)

# Take a look at the dataset
crsp

In [ ]:
# Delete duplicate values in the index (firms with multiple observations in a particular month)
crsp = crsp.loc[~crsp.index.duplicated(keep = False)].copy()

# See how many observations were lost after these steps
n_after4 = len(crsp)
print(n_after3, n_after4, n_after3 - n_after4)

### Create new variables

In [ ]:
# mktcap = prc times shrout
crsp['mktcap'] = crsp['prc'] * crsp['shrout'] 

# print results
crsp[['prc','shrout','mktcap']]

### Compounded returns over past 12 months

In [ ]:
# Start by creating a new variable that equals the gross return that month
crsp['ret12'] = 1 + crsp['ret']    

# Print out what we have so far
crsp[['ret','ret12']] 

In [ ]:
# and every month for the past 11 months...
for i in range(11):
    # ... multiply it with the gross return from that past month
    crsp['ret12'] = crsp['ret12'] * (1 + crsp.groupby(level = 'permno')['ret'].shift(i+1))
    
# Print out what we have so far
crsp[['ret','ret12']] 

In [ ]:
# finally, subtract one to get net returns    
crsp['ret12'] = crsp['ret12'] - 1

# Print out what we have so far
crsp[['ret','ret12']] 

In [ ]:
# Check results for one firm (easier to inspect)
crsp.loc[10000,['ret', 'ret12']]

#### Example
Create a new dataframe called tsla that contains just the price and returns of TSLA. Plot the returns.

In [ ]:
# Extract data for tsla
tsla = crsp.loc[crsp['ticker']=='TSLA', ['prc','ret']]
tsla

#plot
#tsla['ret'].plot();

In [ ]:
# reset the index
tsla.reset_index(inplace=True)
tsla

#set index to mdate
tsla.set_index('mdate',inplace = True)
tsla

#redo plot
tsla['ret'].plot()

### Save new version of the crsp table to a pickle file

In [ ]:
# Save ret and mktcap to a new dataset called "crsp_clean" (in a pickle file) 
crsp.to_pickle('./clean_crsp.pkl.zip')

In [ ]:
# Remember you can read this file with "read_pickle"
crsp = pd.read_pickle('./clean_crsp.pkl.zip')
crsp